# PDF 문서 요약하고 번역하기

In [7]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(
    api_key = api_key
)

In [8]:
import PyPDF2

def read_pdf(file_path):
     with open(file_path, 'rb') as file:
          reader = PyPDF2.PdfReader(file)
          text = ''
          for page in reader.pages:
               text += page.extract_text()
     return text


In [9]:
def summarize_text(text):
     response = client.chat.completions.create(
          model="gpt-4o-mini",
          temperature=0,
          messages=[
               { "role": "system", "content": "다음 텍스트를 요약해줘" },
               { "role": "user", "content": text }
          ],
          max_tokens=150,  # 요약 길이 설정
     )
     return response.choices[0].message.content

In [10]:
def translate_text(text, lang="영어"):
     response = client.chat.completions.create(
          model="gpt-4o-mini",
          temperature=0,
          messages=[
               { "role": "system", "content": f"다음 텍스트를 {lang}로 번역해줘" },
               { "role": "user", "content": text }
          ],
     )
     return response.choices[0].message.content

In [11]:
# 메인 함수
def process_pdf(pdf_file_path):
     # 1. PDF에서 텍스트 추출
     extracted_text = read_pdf(pdf_file_path)

     # 2. GPT를 사용하여 요약
     summary = summarize_text(extracted_text)

     # 3. GPT를 사용하여 번역
     extracted_text = translate_text(summary, lang="영어")
     return (summary, extracted_text)


In [12]:
# PDF 파일 처리
pdf_file_path = 'newspaper.pdf'
summary, translated_summary = process_pdf(pdf_file_path)
print(f"요약: {summary}")
print(f"번역: {translated_summary}")

요약: 한국 경제는 저성장 시대에 접어들며 내년 성장률이 2%를 밑돌 것으로 예상되고, 최악의 경우 1%대에 이를 수 있다는 우려가 커지고 있다. 생산, 소비, 투자 지표가 감소하고 있으며, 수출도 둔화되고 있다. 도널드 트럼프의 관세 정책 등 외부 불확실성이 경제에 부정적인 영향을 미치고 있으며, 구조개혁이 지체되고 있다. 전문가들은 경제 성장을 위해 규제 개혁이 시급하다고 강조하고 있다. 한국의 잠재 성장률도 하락할 것으로 보이며, 이는 장기적인 경제 위기를 초래할 수 있다
번역: The South Korean economy has entered a period of low growth, with next year's growth rate expected to fall below 2%, and concerns are growing that it could drop to the 1% range in the worst-case scenario. Indicators of production, consumption, and investment are declining, and exports are also slowing down. External uncertainties, such as Donald Trump's tariff policies, are negatively impacting the economy, and structural reforms are being delayed. Experts emphasize that regulatory reform is urgently needed for economic growth. South Korea's potential growth rate is also expected to decline, which could lead to a long-term economic crisis.


In [14]:
# 요약한 텍스트를 음성으로 읽어주는 서비스를 구현해보세요.
import pyaudio

def text_to_speech(text):
    p = pyaudio.PyAudio()
    stream = p.open(format=8,
                    channels=1,
                    rate=24_000,
                    output=True)

    with client.audio.speech.with_streaming_response.create(
            model="tts-1",
            voice="alloy",
            input=summary,
            response_format="pcm"
    ) as response:
        for chunk in response.iter_bytes(1024):
            stream.write(chunk)

In [15]:
# PDF 파일 처리
pdf_file_path = 'sports-news.pdf'
summary, translated_summary = process_pdf(pdf_file_path)
print(f"요약: {summary}")
text_to_speech(summary)

요약: KIA 타이거즈의 정해영이 마무리 투수로서 최초로 올스타전 팬 투표에서 최다 득표를 기록했다. 그는 139만6077표를 얻어 2위인 양의지(두산 베어스)를 약 3만3천표 차이로 제치고 팬 투표에서 1위를 차지했다. 정해영은 현재 2승 1패, 20세이브, 평균자책점 2.08의 성적을 기록 중이며, KBO 올스타전은 7월 6일 인천에서 개최된다. 올해 올스타 투표는 총 322만7578표로
